This is a _working_ notebook. See the `ccdControlIntro` notebook for a more narrative how-to. In this notebook, most cells have acquisition, processing, and plotting.

In [1]:
# All system imports
import logging
import time
import clocks
import fitsio

import numpy as np
import scipy
import scipy.optimize as opt

%matplotlib inline
import matplotlib.pyplot as plt


/home/pfs/hostdevel/fullAnaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
cam='b9'

import power.PCM as PCM
reload(PCM)

pcm = PCM.PCM(host='pcm-%s' % (cam))
pcm.logger.setLevel(logging.DEBUG)

In [3]:
import fpga.ccdFuncs as ccdFuncs
reload(ccdFuncs)
import testing.ccdProcedures as ccdTests
reload(ccdTests)

<module 'testing.ccdProcedures' from '/home/pfs/ics_xcu_fpga/python/testing/ccdProcedures.pyc'>

In [4]:
def gobbleFeeOutput():
    while True:
        ret = fee.readResponse()
        print "got: ", ret
        if ret == '':
            break

def feeDown():
    global fee
    fee.powerDown()
    time.sleep(0.25)
    pcm.powerOff('fee')
    time.sleep(0.25)
    
def feeUp0():
    print "feeUp0"
    pcm.powerOn('fee')
    time.sleep(4)

def feeUp1():
    global fee
    print "feeUp1"
    logLevel = 20
    fee = feeMod.FeeControl(logLevel=logLevel)

def feeUp():
    feeUp0()
    feeUp1()
    
def feeBump():
    print "feeBump"

    pcm.powerCmd('fee', False)
    time.sleep(3)
    feeUp0()

def fullBump():
    feeUp1()
    time.sleep(1.5)
    fee.powerDown()
    time.sleep(1)
    pcm.powerOff('fee')
    
def V0():
    feeUp0()
    feeUp1()
    fee.setMode('wipe')
    time.sleep(1)
    fee.setMode('expose')
    time.sleep(1)
    fee.setMode('read')
    ccd.readImage(nrows=40, ncols=100, 
                  clockFunc=pfsClocks, doSave=False)
    fee.setMode('erase')
    time.sleep(1)
#    fee.powerDown()
#    time.sleep(1)
    pcm.powerOff('fee')
    time.sleep(1)
    
def V1():
    feeUp()
    # In erase at this point
    ccdFuncs.fullExposure('dark', ccd=ccd, 
                          expTime=0.5, doSave=False,
                          ncols=300, nrows=10)
    time.sleep(0.5)
    feeDown()
    time.sleep(1)
    
#feeUp1()
#time.sleep(2)
#feeBump()
#print fee.doGet('voltage','54VP')


In [5]:
def calcOffsets(target, current):
    m = np.round((target - current) / 30, 1)
    r = np.round(m * 40.0/57.0)

    return m, r


In [6]:
#fee.powerDown()
pcm.powerOff('fee')

DEBUG:root:sending '~06\r\n'
DEBUG:root:received 'Success'


Success


In [7]:
# FEE connection and initialization.
# Initialize everything to some sane state.
import fee.feeControl as feeMod
reload(feeMod)

doLoad = False # '/home/pfs/feeMain-2015-12-21_01.hex'
doCalib = False

logLevel = 20
if doLoad:
    logLevel = 5
    fee = feeMod.FeeControl(logLevel=logLevel, noPowerup=True)
    pcm.powerOff('fee')
    time.sleep(0.5)

    pcm.powerOn('fee')
    fee.sendImage(doLoad)

    time.sleep(2)
    pcm.powerOff('fee')
    time.sleep(1)
    pcm.powerOn('fee')
    time.sleep(4)
    fee = feeMod.FeeControl(logLevel=logLevel, noPowerup=True)
    fee.powerUp()
else:
    pcm.powerOn('fee')
    time.sleep(3.5)
    fee = feeMod.FeeControl(logLevel=logLevel)
    
if doCalib:
    if not doLoad:
        fee.saveModesOnFee()

    fee.raw('lp,read')
    fee.raw('cal,CDS')
    fee.raw('cal,bias')

    time.sleep(1)
    pcm.powerOff('fee')
    time.sleep(1)
    pcm.powerOn('fee')
    time.sleep(4)
    fee = feeMod.FeeControl(logLevel=logLevel)
    


DEBUG:root:sending '~16\r\n'
DEBUG:root:received 'Success'


Success
SUCCESS
SUCCESS


In [8]:
t0 = time.time()
fee.logger.setLevel(5)
s = fee.statusAsCards()
fee.logger.setLevel(20)
t1 = time.time()
print t1-t0
s

DEBUG:root:sending command :'~rt,all\n':
DEBUG:root:received :~rt,all:
DEBUG:root:received :272.67,272.69,295.96,294.67:


get all enable: 0.00s
get all temps: 1.03s

DEBUG:root:sending command :'~rb,all,ch0\n':
DEBUG:root:received :~rb,all,ch0:
DEBUG:root:received :0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.66,-0.66,0.00:
DEBUG:root:sending command :'~rb,all,ch1\n':
DEBUG:root:received :~rb,all,ch1:
DEBUG:root:received :0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.67,-0.66,0.00:



get all bias: 0.63s

DEBUG:root:sending command :'~rv,all\n':
DEBUG:root:received :~rv,all:
DEBUG:root:received :3.36,3.35,5.21,-5.05,5.11,-5.01,11.85,-11.72,-23.56,50.71:



get all voltage: 0.23s

DEBUG:root:sending command :'~ro,all,ch0\n':
DEBUG:root:received :~ro,all,ch0:
DEBUG:root:received :0.17,0.12,0.12,0.18,0.25,0.19,0.19,0.26:
DEBUG:root:sending command :'~ro,all,ch1\n':
DEBUG:root:received :~ro,all,ch1:
DEBUG:root:received :0.00,-0.08,-0.08,0.03,-0.02,0.02,0.02,0.00:



get all offset: 0.53s
get all preset: 0.00s
2.44384598732


[('serial.FEE', '3060344', ''),
 ('serial.ADC', '3055', ''),
 ('serial.PA0', '194', ''),
 ('serial.CCD0', 'dummy_00', ''),
 ('serial.CCD1', 'dummy_01', ''),
 ('revision.FEE', 'PFS BEE Rev 20151221-02, copyright JHU', ''),
 ('temps.CCD0', '272.67', ''),
 ('temps.CCD1', '272.69', ''),
 ('temps.FEE', '295.96', ''),
 ('temps.PA', '294.67', ''),
 ('bias.ch0.P_on', 0.0, ''),
 ('bias.ch0.P_off', 0.0, ''),
 ('bias.ch0.DG_on', 0.0, ''),
 ('bias.ch0.DG_off', 0.0, ''),
 ('bias.ch0.S_on', 0.0, ''),
 ('bias.ch0.S_off', 0.0, ''),
 ('bias.ch0.SW_on', 0.0, ''),
 ('bias.ch0.SW_off', 0.0, ''),
 ('bias.ch0.RG_on', 0.0, ''),
 ('bias.ch0.RG_off', 0.0, ''),
 ('bias.ch0.OG', 0.0, ''),
 ('bias.ch0.RD', -0.66000000000000003, ''),
 ('bias.ch0.OD', -0.66000000000000003, ''),
 ('bias.ch0.BB', 0.0, ''),
 ('bias.ch1.P_on', 0.0, ''),
 ('bias.ch1.P_off', 0.0, ''),
 ('bias.ch1.DG_on', 0.0, ''),
 ('bias.ch1.DG_off', 0.0, ''),
 ('bias.ch1.S_on', 0.0, ''),
 ('bias.ch1.S_off', 0.0, ''),
 ('bias.ch1.SW_on', 0.0, ''),
 ('bi

In [11]:
# FPGA connection and initialization.
# All the reloads are to make sure that new code gets picked up
import fpga as fpgaMod
#import fpga.ccdFuncs as ccdFuncs
import fpga.nbFuncs as nbFuncs

reload(fpgaMod)
reload(fpgaMod.ccd)
reload(fpgaMod)

ccd = fpgaMod.ccd.CCD(9, 'blue', adc18bit=1)  # adc18bit=2 reads the low bits.
ccd.pciReset()
print "FPGA version: 0x%08x" % (ccd.peekWord(0))

FPGA version: 0xbeefa070


In [12]:
fee.logger.setLevel(20)
#gobbleFeeOutput()
#print fee.device.timeout
for i in range(1):
    ccd1 = fee.sendCommandStr('rt,ccd1')
    ccd0 = fee.sendCommandStr('rt,ccd0')
    print "%s %s" % (ccd0, ccd1)

print fee.sendCommandStr('rt,PA')
print fee.sendCommandStr('rt,FEE')

272.70 272.57
294.67
289.91


In [13]:
%pdb off
fee.logger.setLevel(20)
fee.setMode('read')
t0 = time.time()
feeCards = fee.statusAsCards()
t1 = time.time()
fee.setMode('erase')

print "that took %0.3f s" % (t1-t0)
fee.printStatus()



Automatic pdb calling has been turned OFF
get all enable: 0.00s
get all temps: 1.15s
get all bias: 0.63s
get all voltage: 0.23s
get all offset: 0.53s
get all preset: 0.00s
that took 2.566 s
serial.FEE :  3060344
serial.ADC :  3055
serial.PA0 :  194
serial.CCD0 :  dummy_00
serial.CCD1 :  dummy_01
revision.FEE :  PFS BEE Rev 20151221-02, copyright JHU
temps.CCD0 :  272.59
temps.CCD1 :  272.50
temps.FEE :  296.10
temps.PA :  294.79
bias.ch0.P_on :  -5.03
bias.ch0.P_off :  3.01
bias.ch0.DG_on :  4.99
bias.ch0.DG_off :  4.99
bias.ch0.S_on :  -6.04
bias.ch0.S_off :  3.01
bias.ch0.SW_on :  -6.04
bias.ch0.SW_off :  5.04
bias.ch0.RG_on :  -7.57
bias.ch0.RG_off :  2.01
bias.ch0.OG :  -4.54
bias.ch0.RD :  -11.98
bias.ch0.OD :  -20.97
bias.ch0.BB :  29.43
bias.ch1.P_on :  -5.04
bias.ch1.P_off :  3.01
bias.ch1.DG_on :  4.99
bias.ch1.DG_off :  4.99
bias.ch1.S_on :  -6.04
bias.ch1.S_off :  3.01
bias.ch1.SW_on :  -6.04
bias.ch1.SW_off :  5.04
bias.ch1.RG_on :  -7.57
bias.ch1.RG_off :  2.01
bias.ch1.OG

In [14]:
cmds = 'gb', 'rb'
chans = 'ch0', 'ch1'

fee.logger.setLevel(20)
for mode in 'erase', 'wipe', 'expose', 'read', 'revRead', 'idle':
    fee.setMode(mode)
    time.sleep(0.25)
    
    print "======== %s" % mode
    
    for k in fee.allKeys('bias'):
        print "%-8s " % k,
        for chan in chans:
            for cmd in cmds:
                v = fee.sendCommandStr('%s,%s,%s' % (cmd, k, chan))
                print "%6s " % (v),
            print "    ",
        print

======== erase
P_on        6.00    5.99         6.00    5.99      
P_off       6.00    5.99         6.00    5.99      
DG_on       6.00    5.99         6.00    5.99      
DG_off      6.00    5.99         6.00    5.99      
S_on        6.00    5.99         6.00    5.99      
S_off       6.00    5.99         6.00    5.99      
SW_on       6.00    5.99         6.00    5.99      
SW_off      6.00    6.05         6.00    6.05      
RG_on       6.00    6.00         6.00    6.00      
RG_off      6.00    6.00         6.00    6.00      
OG          6.00    6.00         6.00    6.00      
RD        -12.00  -11.98       -12.00  -11.99      
OD         -5.00   -4.98        -5.00   -4.98      
BB          0.20    0.00         0.20    0.00      
======== wipe
P_on       -5.00   -5.03        -5.00   -5.04      
P_off       3.00    3.01         3.00    3.01      
DG_on       5.00    4.99         5.00    4.99      
DG_off      5.00    4.99         5.00    4.99      
S_on       -6.00   -6.04        -6.

In [ ]:
fee.logger.setLevel(5)
fee.setMode('read')

print fee.raw('gb,RD,ch1')
print fee.raw('rb,RD,ch1')
print fee.raw('gb,all,ch1')
print fee.raw('rb,all,ch1')

fee.logger.setLevel(20)


In [ ]:
print fee.raw

In [ ]:
if False:
    fee.logger.setLevel(5)
    fee.device.setTimeout(2.0)
    print fee.saveModesOnFee()
    fee.device.setTimeout(0.5)
    fee.logger.setLevel(20)


In [ ]:
if True:
    print fee.doGet('serial', 'FEE')
    fee.unlockConfig()
    fee.setSerial('CCD0', 'dummy_0')
    fee.setSerial('CCD1', 'dummy_1')
    fee.lockConfig()
print fee.doGet('serial', 'FEE')
print fee.doGet('serial', 'ADC')
print fee.doGet('serial', 'PA0')
print fee.doGet('serial', 'CCD0')
print fee.doGet('serial', 'CCD1')


In [ ]:
# Load the clock tables. pfsClocks is used for each readout.
import clocks
import clocks.read
reload(clocks.read)
reload(clocks)

pfsClocks = clocks.read.readClocks
pre, pix, post = pfsClocks()


In [ ]:
def setGains(_lev):
    refSlope = "p"
    masterSlope = "n"

    _ref = _lev
    _master = _
    defOffsets = np.array([_lev]*8)
    fee.setOffsets(np.arange(8), defOffsets, leg=masterSlope)

    defOffsets = np.array([_lev*(-55.0/-66)]*8)
    fee.setOffsets(np.arange(8), defOffsets, leg=refSlope)


def gainTest1(offset, nrows=200, clockFunc=pfsClocks, doUnwrap=65000, statCols=slice(30,-30)):
        setGains(offset)
        
        argDict = dict(everyNRows=100, ampList=amps, ccd=ccd)                                                                                           

        
        im, files = ccd.readImage(nrows=nrows, rowFunc=ccdFuncs.rowStats, rowFuncArgs=argDict, doSave=False,                                        
                                  clockFunc=clockFunc)                                                                                              
        if doUnwrap is not False:                                                                                                                   
            im = im.astype('i4')                                                                                                                    
            hi_w = im > doUnwrap                                                                                                                    
            if hi_w.sum() > 0:                                                                                                                      
                print "!!!! unwraping %d pixels !!!!" % (hi_w.sum())                                                                                
                im[hi_w] -= 65535                                                                                                                   
                                                                                                                                                    
        newLevels, devs = nbFuncs.ampStats(im, statCols, ccd=ccd)                                                                                           
        print "means(%0.3f): %s" % (offset, nbFuncs.fmtArr(newLevels))                                                                                      
        print "devs (%0.3f): %s" % (offset, nbFuncs.fmtArr(devs))      
        
        return newLevels, devs


In [ ]:
%pdb off

reload(ccdFuncs)
reload(nbFuncs)

# fee.setMode('read')
doGainCurve = True
if doGainCurve:
    fee.zeroOffsets()
    stepSize = 19.9 * 4
    statCols = slice(100,-100)
    amps = np.arange(8)
    noffsets, nlevels = nbFuncs.gainCurve(ccd, fee, amps=amps, stepSize=stepSize, leg='n', offLimit=400,
                                          statCols=statCols, doUnwrap=65000, clockFunc=pfsClocks, nrows=300)
    poffsets, plevels = nbFuncs.gainCurve(ccd, fee, amps=amps, stepSize=-stepSize, leg='p', offLimit=400,
                                          statCols=statCols, doUnwrap=65000, clockFunc=pfsClocks, nrows=300)
    
    print
    nfits = nbFuncs.plotGains(noffsets, nlevels, amps=amps)
    print
    pfits = nbFuncs.plotGains(poffsets, plevels, amps=amps)
    
fee.zeroOffsets()

In [ ]:
%pdb off
reload(ccdFuncs)
reload(nbFuncs)

allAmps = np.arange(8)
badAmps = []

dummyCcd = False
if not dummyCcd:
    goodAmps = np.arange(8)

    startStep = 5
    levels = 1000
else:
    goodAmps = np.arange(4) + 4
    
    startStep = 10
    levels = np.zeros(8)
    levels[4:] = [10000,25000,20000,15000]

ncols=300

doTune = False
if doTune:
    offs, devs, gains = nbFuncs.tuneLevels(ccd, fee, levels=levels, adjOffset=5, 
                                           amps=goodAmps,
                                           statCols=slice(2,None), 
                                           maxLoops=10, nrows=200, 
                                           startStep=startStep, 
                                           startOffset=startStep,
                                           sigTol=4, 
                                           clockFunc=pfsClocks, 
                                           legs='np', # useGains=nfits[:,0], 
                                           doZero=True, doUnwrap=65000)

    defOffsets = offs


In [ ]:
refSlope = "p"
masterSlope = "n"
# fee.zeroOffsets()

current = np.array([6204.921, 5003.009, 3296.557, 2548.498, 6141.049, 5045.979, 6259.639, 4928.726])
current = np.array([6256.143, 5016.784, 3245.525, 2541.481, 6069.705, 5083.763, 3980.821, 2596.544])
current = np.array([6198.592, 5000.811, 3932.805, 2616.125, 6143.811, 5047.019, 6265.157, 4926.674])
current = np.array([6257.553, 5016.021, 3906.896, 2612.281, 6074.105, 5082.719, 3979.107, 2592.892])

zeros = np.zeros(8)
m, r = calcOffsets(1000,current)
print repr(m)
print repr(r)

fee.logger.setLevel(20)
masterOffsets = m # np.array([0]*8)
refOffsets = r # np.array([0]*8)
fee.setOffsets(np.arange(8), masterOffsets, leg=masterSlope)
fee.setOffsets(np.arange(8), refOffsets, leg=refSlope)
fee.logger.setLevel(20)

fee.getCommandStatus(fee.commands['offset'])

In [ ]:
fee.zeroOffsets()

In [ ]:
feeDown()
V1()
time.sleep(2)
feeUp()

In [ ]:
reload(ccdFuncs)
reload(nbFuncs)

goodAmps = np.arange(8) #[3,4,5,6, 7] # np.arange(8)
badAmps = []

nrows = 400          # How many rows to read out.
ncols = 400
selectRows = slice(20,nrows-20)
selectCols = slice(20,-5)
pcols = np.arange(32,ncols-64)    # Which cols to plot
statCols = np.arange(40,ncols-64) # Which cols to use for stats
prow = (selectRows.stop+1)/2
prows = [prow-100,prow,prow+100]       # which rows to plot
fftAmps = []

figWidth = 12

argDict = dict(everyNRows=300, ampList=goodAmps, cols=statCols, ccd=ccd)

siggenAmp = None
if siggenAmp is None:
    try:
        switch.setCoils(off=range(16))
    except:
        pass
else:
    switch.chooseCoil(siggenAmp)

comment = 'pure test'
doFile = False # '/data/pfs/2015-12-09/PFSA00706091.fits'
doSave = True
doWipe = cam != 'b9'
if doFile:
    im = fitsio.read(doFile)
else:
#    im, files = ccd.readImage(nrows=nrows, ncols=ncols, 
#                              rowFunc=ccdFuncs.rowStats, rowFuncArgs=argDict, 
#                              clockFunc=pfsClocks, doSave=doSave,
#                              comment=comment, addCards=feeCards)
#    fee.setMode('erase')
    im, files = ccdFuncs.fullExposure('bias', ccd=ccd,
                                      nrows=nrows, ncols=ncols, 
#                                 rowFunc=ccdFuncs.rowStats, rowFuncArgs=argDict, 
                                      clockFunc=pfsClocks, doSave=doSave,
                                      doWipe=doWipe,
                                      comment=comment)
    fee.setMode('idle')

if selectRows is not None:
    im = im[selectRows]
if selectCols is not None:
    im = im[:,selectCols]
    
im = im.astype('i4')
if False:
    hi_w = np.where(im > 63000)
    im[hi_w] -= 65535

if siggenAmp is None:
    amp4Mask = ccd.ampidx(4, im)
else:
    amp4Mask = ccd.ampidx(siggenAmp, im)

#amp6Mask = ccd.ampidx(6, im)
#im[:, amp6Mask] -= im[:, amp6Mask].mean()
#im[:, amp6Mask] += im[:, ccd.ampidx(4, im)].mean()

means,devs = nbFuncs.ampStats(im, ccd=ccd, cols=statCols)
print "means:", nbFuncs.fmtArr(means, format="%0.3f")
print "devs: ", nbFuncs.fmtArr(devs, format="%0.3f")
med4 = np.median(im[prow, amp4Mask])
max4 = np.max(im[prow, amp4Mask])
min4 = np.min(im[prow, amp4Mask])

inject = 0.3
print "peak: ", med4, min4, max4, max4-med4, (max4-med4)/inject, inject/(max4-med4)

# print "shape: %s dtype: %s" % (str(im.shape), im.dtype)
    
if len(files) > 0:
    path = files[0]
    imName = '/'.join(path.split('/')[-2:])
    print "file: %s" % (path)

#focusAmp = 0             # Which amp to focus on
#focusMask = ccd.ampidx(focusAmp, im)[pcols]

# plot one or more rows across the entire image
nbFuncs.plotRows(im, prows, imName=imName, figWidth=figWidth, pixRange=200 if siggenAmp is not None else None)

# plot one row for each the "good" amps
peaks = None # np.arange(1, ncols, 50)
if siggenAmp is not None:
    channel = siggenAmp/4
    amps0 = [siggenAmp]
    amps1 = goodAmps[np.where((goodAmps/4 == channel) & (goodAmps != siggenAmp))]
    amps2 = goodAmps[np.where(goodAmps/4 != channel)]

    nbFuncs.plotAmps(im, amps=amps0, row=prow, cols=pcols, 
                 plotOffset=10, fig=None, figWidth=figWidth, 
                 peaks=peaks, clipPeaks=True)

    nbFuncs.plotAmps(im, amps=amps1, row=prow, cols=pcols, 
                 plotOffset=100, fig=None, figWidth=figWidth, 
                 peaks=peaks, clipPeaks=True)

    nbFuncs.plotAmps(im, amps=amps2, row=prow, cols=pcols, 
                 plotOffset=10, fig=None, figWidth=figWidth)
else:
    nbFuncs.plotAmps(im, amps=goodAmps, row=prow, cols=pcols, 
                     plotOffset=10, fig=None, figWidth=figWidth, 
                     peaks=peaks, clipPeaks=True)

# f.axes[0].set_ylim(-50, 150)

# plot any bad amps individually
for a in badAmps:
    nbFuncs.plotAmps(im, amps=[a], row=prow, cols=pcols, figWidth=figWidth)

# show a grid of histograms.
hists = nbFuncs.ampHistGrid(im, ccd, cols=pcols, histRange=20, figWidth=figWidth)

# show a grid of raw amp images.
nbFuncs.rawAmpGrid(im, ccd, cols=pcols, figWidth=figWidth, showFfts=True)

fftAmps = np.arange(8) # [1,2,5] # [5,6]
if len(fftAmps) > 0:    
    fftFig, pl = plt.subplots(nrows=len(fftAmps),
                              figsize=(figWidth, figWidth/4 * len(fftAmps)))
    for a in fftAmps:
        fftMask = ccd.ampidx(a, im)[pcols]
        ampSeg = im[prow,fftMask]
        freqs, fft, peak_ii = ccdFuncs.plotTopPeriods(ampSeg, plot=pl[a], topN=3)


#fig.colorbar(pp)
#plt.tight_layout()